In [2]:
# import packages
import pandas as pd
import numpy as np
import re

In [ ]:
#create dataframes

course_path = 'C:/Users/ywa/Documents/Data/Moodle Query/June2025/Course - Category - Enrollments.csv' 
cert_path = 'C:/Users/ywa/Documents/Data/Moodle Query/June2025/Course - Certificates.csv'

course_df = pd.read_csv(course_path)
cert_info = pd.read_csv(cert_path)
#course_df.head()
cert_info.head()


In [ ]:
# groupby cert_info by courseID
participation_key = r'teilnahmebestätigung|participation'
achievement_key = r'leistungsnachweis|leistungsnachweise|achivement|record of achievement|zertifikat|certificate|transcript of records'

cert_info['name_lower'] = cert_info['certificateName'].str.lower()

def classify_cert(name):
    if pd.isna(name):
        return None
    name = name.lower()
    if re.search(participation_key, name):
        return 'CoP'
    if re.search(achievement_key, name):
        return 'RoA'
    else:
        return None

cert_info['cert_type'] = cert_info['name_lower'].apply(classify_cert)

cert_info_filtered = cert_info[cert_info['cert_type'].notna()]

cert_df = (cert_info_filtered.groupby(['courseID', 'cert_type'])['amount'].sum().unstack('cert_type').reset_index())

cert_df.fillna(0, inplace = True)

cert_df[['CoP', 'RoA']] = cert_df[['CoP', 'RoA']].astype(int)

cert_df.columns = ['courseID', 'comfirmation_of_participation', 'records_of_achievement']

print(cert_df.shape[0])

cert_df.head()


In [ ]:
# merge two dataframes
course_df = course_df[['courseID', 'courseName', 'categoryName', 'enrollments']]

print(len(course_df))
print(len(cert_df))

merged_df = course_df.merge(cert_df, on = 'courseID', how = 'left')

print(len(merged_df))

merged_df.head()

In [ ]:
# import base table
base_table_path = 'C:/Users/ywa/Documents/Data/Moodle Query/June2025/KIC-course completion rate.csv'

base_table = pd.read_csv(base_table_path, sep = ';', encoding = 'utf8')

base_table.head()


In [ ]:
# merge the cert data with the base table
base_merge = merged_df.merge(base_table, on = 'courseID', how = 'outer')
base_merge 


In [ ]:
# export data for spreadsheet check
#merged_df.to_excel('C:/Users/ywa/Documents/Data/Moodle Query/June2025/MoodleEnrole.xlsx', index = False)
base_merge.to_excel('C:/Users/ywa/Documents/Data/Moodle Query/June2025/MoodleEnrole.xlsx', index = False)